# IMPORTANT 
Make sure you've imported the [environment.yml](https://github.com/Snowflake-Labs/sfguide-intro-to-machine-learning-with-snowflake-ml-for-python/blob/main/notebooks/environment.yml) file provided in the git repo on the left sidebar.

This will ensure if you have the right packages needed to run this Notebook.

## 2. ML Feature Transformations

- In this notebook, we will walk through a few transformations that are included in the Snowpark ML Preprocessing API. 
- We will also build a preprocessing pipeline to be used in the ML modeling notebook.

***Note: All feature transformations using Snowpark ML are distributed operations in the same way that Snowpark DataFrame operations are.***

### Import Libraries

In [ ]:
# Snowpark for Python
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import DecimalType

# Snowpark ML
import snowflake.ml.modeling.preprocessing as snowml
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.metrics.correlation import correlation

# Data Science Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Misc
import json
import joblib

# warning suppresion
import warnings; warnings.simplefilter('ignore')

### Establish Secure Connection to Snowflake

Notebooks establish a Snowpark Session when the notebook is attached to the kernel. Let's use that Session object to validate our connection.

In [ ]:
-- Using Warehouse, Database, and Schema created during Setup
USE WAREHOUSE ML_HOL_WH;
USE DATABASE ML_HOL_DB;
USE SCHEMA ML_HOL_SCHEMA;

In [ ]:
session = get_active_session()

# Add a query tag to the session.
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"e2e_ml_snowparkpython", 
                     "version":{"major":1, "minor":0,},
                     "attributes":{"is_quickstart":1}}
session

### Data Loading

In [ ]:
# First, we read in the data from a Snowflake table into a Snowpark DataFrame
# **Change this only if you named your table something else in the data ingest notebook **
diamonds_df = session.table("DIAMONDS")

### Feature Transformations

We will illustrate a few of the transformation functions here, but the rest can be found in the [documentation](https://docs.snowflake.com/LIMITEDACCESS/snowflake-ml-preprocessing).

##### Let's use the `MinMaxScaler` to normalize the `CARAT` column.

In [ ]:
# Normalize the CARAT column
snowml_mms = snowml.MinMaxScaler(input_cols=["CARAT"], output_cols=["CARAT_NORM"])
normalized_diamonds_df = snowml_mms.fit(diamonds_df).transform(diamonds_df)

# Reduce the number of decimals
new_col = normalized_diamonds_df.col("CARAT_NORM").cast(DecimalType(7, 6))
normalized_diamonds_df = normalized_diamonds_df.with_column("CARAT_NORM", new_col)

normalized_diamonds_df

##### Let's use the `OrdinalEncoder` to transform `COLOR` and `CLARITY` from categorical to numerical values so they are more meaningful.

In [ ]:
# Encode CUT and CLARITY preserve ordinal importance
categories = {
    "CUT": np.array(["IDEAL", "PREMIUM", "VERY_GOOD", "GOOD", "FAIR"]),
    "CLARITY": np.array(["IF", "VVS1", "VVS2", "VS1", "VS2", "SI1", "SI2", "I1", "I2", "I3"]),
}
snowml_oe = snowml.OrdinalEncoder(input_cols=["CUT", "CLARITY"], output_cols=["CUT_OE", "CLARITY_OE"], categories=categories)
ord_encoded_diamonds_df = snowml_oe.fit(normalized_diamonds_df).transform(normalized_diamonds_df)

# Show the encoding
print(snowml_oe._state_pandas)

ord_encoded_diamonds_df

##### Let's use the `OneHotEncoder` to transform the categorical columns to numerical columns.

This is more for illustration purposes. Using the OrdinalEncoder makes more sense for the diamonds dataset since `CARAT`, `COLOR`, and `CLARITY` all follow a natural ranking order.

In [ ]:
# Encode categoricals to numeric columns
snowml_ohe = snowml.OneHotEncoder(input_cols=["CUT", "COLOR", "CLARITY"], output_cols=["CUT_OHE", "COLOR_OHE", "CLARITY_OHE"])
transformed_diamonds_df = snowml_ohe.fit(ord_encoded_diamonds_df).transform(ord_encoded_diamonds_df)

np.array(transformed_diamonds_df.columns)

##### Finally, we can also build out a full preprocessing `Pipeline`.

This will be useful for both the ML training & inference steps to have standarized feature transformations.

In [ ]:
# Categorize all the features for processing
CATEGORICAL_COLUMNS = ["CUT", "COLOR", "CLARITY"]
CATEGORICAL_COLUMNS_OE = ["CUT_OE", "COLOR_OE", "CLARITY_OE"] # To name the ordinal encoded columns
NUMERICAL_COLUMNS = ["CARAT", "DEPTH", "TABLE_PCT", "X", "Y", "Z"]

categories = {
    "CUT": np.array(["IDEAL", "PREMIUM", "VERY_GOOD", "GOOD", "FAIR"]),
    "CLARITY": np.array(["IF", "VVS1", "VVS2", "VS1", "VS2", "SI1", "SI2", "I1", "I2", "I3"]),
    "COLOR": np.array(['D', 'E', 'F', 'G', 'H', 'I', 'J']),
}

In [ ]:
# Build the pipeline
preprocessing_pipeline = Pipeline(
    steps=[
            (
                "OE",
                snowml.OrdinalEncoder(
                    input_cols=CATEGORICAL_COLUMNS,
                    output_cols=CATEGORICAL_COLUMNS_OE,
                    categories=categories,
                )
            ),
            (
                "MMS",
                snowml.MinMaxScaler(
                    clip=True,
                    input_cols=NUMERICAL_COLUMNS,
                    output_cols=NUMERICAL_COLUMNS,
                )
            )
    ]
)

PIPELINE_FILE = '/tmp/preprocessing_pipeline.joblib'
joblib.dump(preprocessing_pipeline, PIPELINE_FILE) # We are just pickling it locally first

transformed_diamonds_df = preprocessing_pipeline.fit(diamonds_df).transform(diamonds_df)
transformed_diamonds_df

In [ ]:
# You can also save the pickled object into the stage we created earlier for deployment
session.file.put(PIPELINE_FILE, "@ML_HOL_ASSETS", overwrite=True)

### Data Exploration

Now that we've transformed our features, let's calculate the correlation using Snowpark ML's `correlation()` function between each pair to better understand their relationships.

*Note: Snowpark ML's pearson correlation function returns a Pandas DataFrame*

In [ ]:
# corr_diamonds_df = correlation(df=transformed_diamonds_df)
# corr_diamonds_df # This is a Pandas DataFrame

In [ ]:
# # Generate a mask for the upper triangle
# mask = np.triu(np.ones_like(corr_diamonds_df, dtype=bool))

# # Create a heatmap with the features
# plt.figure(figsize=(7, 7))
# heatmap = sns.heatmap(corr_diamonds_df, mask=mask, cmap="YlGnBu", annot=True, vmin=-1, vmax=1)

We note that `CARAT` and `PRICE` are highly correlated, which makes sense! Let's take a look at their relationship a bit closer.

*Note: You will have to convert your Snowpark DF to a Pandas DF in order to use matplotlib & seaborn.*

In [ ]:
# Set up a plot to look at CARAT and PRICE
counts = transformed_diamonds_df.to_pandas().groupby(['PRICE', 'CARAT', 'CLARITY_OE']).size().reset_index(name='Count')

fig, ax = plt.subplots(figsize=(20, 20))
plt.title('Price vs Carat', fontsize=28)
ax = sns.scatterplot(data=counts, x='CARAT', y='PRICE', size='Count', hue='CLARITY_OE', markers='o')
ax.grid(axis='y')

# The relationship is not linear - it appears exponential which makes sense given the rarity of the large diamonds
sns.move_legend(ax, "upper left")
sns.despine(left=True, bottom=True)

In the next notebook, we will look at how you can train an XGBoost model with the diamonds dataset using the Snowpark ML Modeling API.